# Lecture 4.6 - Basics of cleaning messy text files 
## Part 2 - Grouping blocks of data and extracting information

In this lecture, we will go over a number of cases of messy data, and how to use Python to fix these problems.  This includes

1. Removing unwanted lines.
2. Parsing lines with regular expressions.
3. Working with data blocks spread across multiple lines.

## Reading in current progress

In [1]:
with open('911_Deaths_Grouped.csv') as f:
    content = f.read()
content[:500]

"Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.\nEdelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\nMarie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.\nAndrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.\nVincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.\nLaurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.\nAlona Abraham, 3"

In [2]:
grouped_lines = content.split('\n')
grouped_lines

antor Fitzgerald, World Trade Center.',
 'Anthony J. Fallone, Jr., 39, New York City, Cantor Fitzgerald, World Trade Center.',
 'Dolores Brigitte Fanelli, 38, Farmingville, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Robert John Fangman, 33, Chelsea, Mass., Flight Crew, United 175, World Trade Center.',
 'John Joseph Fanning, 54, West Hempstead, N.Y., New York City Fire Department, World Trade Center.',
 'Kathleen Anne Faragher, 33, Risk Waters Group conference attendee from Janus Capital Group, World Trade Center.',
 'Thomas James Farino, 37, Bohemia, N.Y., New York City Fire Department, World Trade Center.',
 'Nancy C. Doloszycki Farley, 45, Jersey City, N.J., Reinsurance Solutions, World Trade Center.',
 'Paige Marie Farley-Hackel, 46, Newton, Mass., Passenger, United 11, World Trade Center.',
 'Elizabeth Ann Farmer, 62, Cantor Fitzgerald contractor, World Trade Center.',
 'Douglas Jon Farnum, 33, Brooklyn, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.'

## Preprocessing 

Below I have transfered over the preprocessing functions and applied them to the data.

In [3]:
# Imports
from composable import pipeable
from composable.strict import map

In [4]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))

In [5]:
(grouped_lines
>> map(add_missing_period)
>> map(fix_world_trade)
)

antor Fitzgerald, World Trade Center.',
 'Anthony J. Fallone, Jr., 39, New York City, Cantor Fitzgerald, World Trade Center.',
 'Dolores Brigitte Fanelli, 38, Farmingville, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Robert John Fangman, 33, Chelsea, Mass., Flight Crew, United 175, World Trade Center.',
 'John Joseph Fanning, 54, West Hempstead, N.Y., New York City Fire Department, World Trade Center.',
 'Kathleen Anne Faragher, 33, Risk Waters Group conference attendee from Janus Capital Group, World Trade Center.',
 'Thomas James Farino, 37, Bohemia, N.Y., New York City Fire Department, World Trade Center.',
 'Nancy C. Doloszycki Farley, 45, Jersey City, N.J., Reinsurance Solutions, World Trade Center.',
 'Paige Marie Farley-Hackel, 46, Newton, Mass., Passenger, United 11, World Trade Center.',
 'Elizabeth Ann Farmer, 62, Cantor Fitzgerald contractor, World Trade Center.',
 'Douglas Jon Farnum, 33, Brooklyn, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.'

In [6]:
# For convenience I will give these a name
prepped_lines = (grouped_lines 
                >> map(add_missing_period)
                >> map(fix_world_trade)
                )
prepped_lines

antor Fitzgerald, World Trade Center.',
 'Anthony J. Fallone, Jr., 39, New York City, Cantor Fitzgerald, World Trade Center.',
 'Dolores Brigitte Fanelli, 38, Farmingville, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Robert John Fangman, 33, Chelsea, Mass., Flight Crew, United 175, World Trade Center.',
 'John Joseph Fanning, 54, West Hempstead, N.Y., New York City Fire Department, World Trade Center.',
 'Kathleen Anne Faragher, 33, Risk Waters Group conference attendee from Janus Capital Group, World Trade Center.',
 'Thomas James Farino, 37, Bohemia, N.Y., New York City Fire Department, World Trade Center.',
 'Nancy C. Doloszycki Farley, 45, Jersey City, N.J., Reinsurance Solutions, World Trade Center.',
 'Paige Marie Farley-Hackel, 46, Newton, Mass., Passenger, United 11, World Trade Center.',
 'Elizabeth Ann Farmer, 62, Cantor Fitzgerald contractor, World Trade Center.',
 'Douglas Jon Farnum, 33, Brooklyn, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.'

## Regular expression from lab 2

Below I have attempted to combine all of the regular expressions from lab 2

In [7]:
import re
line_parts = re.compile('^(.+), (\?\?|\d{1,3}),(.*?)( Passenger,| Flight Crew,)?( United \d{2,3},| American \d{2,3},)?( World Trade Center| Pentagon| Shanksville, Pa)(, died \d{1,2}/\d{1,2}/\d{1,2})?\.$')

In [8]:
prepped_lines[2402]

'Jesus Sanchez, 45, Flight Crew, United 175, World Trade Center.'

In [9]:
line_parts.search(prepped_lines[2402]).groups()

('Jesus Sanchez',
 '45',
 '',
 ' Flight Crew,',
 ' United 175,',
 ' World Trade Center',
 None)

#### Always check for non-matches

In [10]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

## Verbose regular expressions

**Pros:**
* Spread over multiple lines
* Allow comments

**Cons:**
* Ignore white space outside `()`
* Require escaping spaces `\ `

In [11]:
# Without Using VERBOSE 
regex_email = re.compile(r'^([a-z0-9_\.-]+)@([0-9a-z\.-]+)\.([a-z\.]{2, 6})$')

In [12]:
# Using VERBOSE 
regex_email = re.compile(r""" 
                        ^([a-z0-9_\.-]+)			 # local Part 
                        @							 # single @ sign 
                        ([0-9a-z\.-]+)			 	 # Domain name 
                        \.						 	 # single Dot . 
                        ([a-z]{2,6})$				 # Top level Domain 
                        """,re.VERBOSE)

## Another example.

This example, from the Python docs, shows how to space out an OR section across multiple lines.

In [13]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

## Cleaning up our regular expr

<h2> <font color="red"> Exercise 4.6.1 - Clean up the regular expression </font> </h2>

To clean up the regular expression, 

1. Replace all spaces with `\ ` or `\s` (I prefer the second)
2. Turn the string into a multi-line string.
3. Spread the parts over many lines
4. Add comments.

In the following strucutre, spaces are reconginzed as formatting, not as actualt regex characters. At a minimun, the spaces need to be replaces with at least \ 

In [14]:
line_parts2 = re.compile(r"""

^(.+),\s        #front part
    (
    \?\?|\d{1,3}),(.*?)(\sPassenger,|   #may be a passenger
    \sFlight Crew,                      #or flight crew
    )?                                  #or neither

    (
        \sUnited\s\d{2,3},|             #may have been on an american flight
        \sAmerican\s\d{2,3},            #a United Flight
    )?                                  #or neither
    ( 
        \sWorld\sTrade\sCenter|         #could have died at WTC
        \sPentagon|                     #Pentagon
        \sShanksville,\sPa              #or PN
    )
    (
        ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2})?\.$ #grab trailing details if they exist
    )
    
""",re.VERBOSE
)

## Progress so far

In [15]:
# Imports
from composable import pipeable
from composable.strict import map

In [16]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [17]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
# New
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))

In [18]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [72]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines[0:30]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anthony Ac

## Pulling out and cleaning up names

Sometimes it is useful to pull the various columns apart and clean them up separately.  To illustrate, will will pull out and clean up the names. We can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [20]:
from toolz.curried import get

In [71]:
(split_lines
>> map(get(0))
)[0:30]

['Gordon M. Aamoth, Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley, Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar, Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

Now we can clean up these name by removing commas.

In [73]:
remove_commas = lambda s: s.replace(',', '')

(split_lines
>> map(get(0))
>> map(remove_commas)
)[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

## Pulling out and cleaning up ages

NExt, we will pull out and clean the ages.  In this case, we should replace the missing values, currently `'??'`, to blanks.

In [70]:
remove_quest_mark = lambda s: s.replace('??', '')

(split_lines
>> map(get(1))
>> map(remove_quest_mark)
)[0:30]

[' 32',
 ' 54',
 ' 49',
 ' 37',
 ' 40',
 ' 37',
 ' 30',
 ' 55',
 ' 42',
 ' 38',
 ' 29',
 ' 37',
 ' 28',
 ' 61',
 ' 25',
 ' 51',
 ' 62',
 ' 28',
 ' 22',
 ' 36',
 ' 48',
 ' 32',
 ' 37',
 ' 36',
 ' 37',
 ' 35',
 ' 46',
 ' 30',
 ' 43',
 ' 74']

## Progress so far

In [24]:
# Imports
from composable import pipeable
from composable.strict import map

In [25]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [26]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
# New
remove_commas = lambda s: s.replace(',', '')
remove_quest_mark = lambda s: s.replace('??', '')

In [27]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [75]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines[0:5]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  '')]

In [67]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

In [66]:
ages =  (split_lines
        >> map(get(1))
        >> map(remove_quest_mark)
        )
ages[0:30]

[' 32',
 ' 54',
 ' 49',
 ' 37',
 ' 40',
 ' 37',
 ' 30',
 ' 55',
 ' 42',
 ' 38',
 ' 29',
 ' 37',
 ' 28',
 ' 61',
 ' 25',
 ' 51',
 ' 62',
 ' 28',
 ' 22',
 ' 36',
 ' 48',
 ' 32',
 ' 37',
 ' 36',
 ' 37',
 ' 35',
 ' 46',
 ' 30',
 ' 43',
 ' 74']

<h2> <font color="red"> Exercise 4.6.2 - Separating and cleaning other columns. </font> </h2>

To clean up the following columns 

1. Grab the date of death and replace the missing values with `9/11/2001`
2. Grab the locations (e.g. `World Trade Center`) and remove the comma from `'Shanksville, Pa.`
3. Grab the flights.
4. Grab the passenger status.

**Note:** Be sure to strip whitespace from all of them.

In [74]:

# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))

# New
remove_commas = lambda s: s.replace(',', '')
remove_quest_mark = lambda s: s.replace('??', '')
replace_blank_death = lambda s: s.replace('', '9/11/2001')
replace_01 = lambda s: s.replace('01', '2001')

(split_lines
>> map(get(6))
)[0:30]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## Grabbing the troubling bit

We have made significant progress, but still need to work on the third entry, which contains the hometown and employment information.  Again, we can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [32]:
troubling_bit = (split_lines
                >> map(get(2))
                )
troubling_bit

nt,',
 " Summit, N.J., Sandler O'Neill + Partners,",
 " Rockville Centre, N.Y., Sandler O'Neill + Partners,",
 ' Rutherford, N.J., Aon Corporation,',
 " New York City, Sandler O'Neill + Partners,",
 ' Carr Futures, Inc.,',
 ' Jersey City, N.J., Cantor Fitzgerald,',
 ' Glen Rock, N.J., Chuo Mitsui Trust and Banking Company, Ltd.,',
 ' Woodstock, N.Y., Fiduciary Trust Company International,',
 ' Summit Security Services, Inc.,',
 ' Wilmot, N.H.,',
 ' Glen Gardner, N.J., Cantor Fitzgerald,',
 ' Port Washington, N.Y., Risk Waters Group,',
 ' Staten Island, N.Y., New York City Fire Department,',
 ' Scarsdale, N.Y., Cantor Fitzgerald,',
 '',
 ' Manasquan, N.J., Cantor Fitzgerald,',
 ' Princeton Junction, N.J., Euro Brokers,',
 ' Staten Island, N.Y., New York City Fire Department,',
 ' Cantor Fitzgerald,',
 ' Norwalk, Conn., Aon Corporation visitor,',
 ' Boston, Mass. and Paris, France,',
 ' Staten Island, N.Y., Cantor Fitzgerald,',
 ' Santa Monica, Calif.,',
 ' Medford, N.Y., New York City P

## Progressively filtering out states

We will start by matching two of the most common states, NY and NJ.

In [33]:
state = re.compile(', (N\.Y\.|N\.J\.),?')
# Rows that match
[(l, state.search(l)) for l in troubling_bit]

y Fire Department,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Middletown, N.J., Aon Corporation,',
  <re.Match object; span=(11, 18), match=', N.J.,'>),
 (' Jersey City, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(12, 19), match=', N.J.,'>),
 (' Marsh&McLennan Companies, Inc.,', None),
 (' Brooklyn, N.Y., Aon Corporation,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Cedar Grove, N.J., Windows on the World visitor,',
  <re.Match object; span=(12, 19), match=', N.J.,'>),
 (' Aon Corporation,', None),
 (' Marsh&McLennan Companies, Inc.,', None),
 (' New York City Fire Department,', None),
 (' South Huntington, N.Y., New York City Police Department,',
  <re.Match object; span=(17, 24), match=', N.Y.,'>),
 (' Cantor Fitzgerald,', None),
 (' North Brunswick, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(16, 23), match=', N.J.,'>),
 (' United States Naval Reserve,', None),
 (' Fiduciary Trust Company International,', None),
 (' Flushing, N.Y., Wi

and inspecting all rows that don't match for additional states or problems

In [34]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

9, ' Arlington, Va., United States Navy Civilian,'),
 (1210, ' Fred Alger Management, Inc.,'),
 (1216, ' New York, Keefe, Bruyette&Woods, Inc.,'),
 (1217, ' Stamford, Conn., Marsh&McLennan Companies, Inc.,'),
 (1222, ' New York City, Marsh&McLennan, Advantage Security,'),
 (1223, ''),
 (1224, ' Norwalk, Conn., Euro Brokers,'),
 (1227, ' Springfield, Va., United States Army Civilian,'),
 (1229, ' Burke, Va., United States Army,'),
 (1230, ' Lake Ridge, Va., Defense Intelligence Agency,'),
 (1231, ' Norwalk, Conn., Thomson Financial/Vestek,'),
 (1234, ' New Jersey, Cantor Fitzgerald,'),
 (1235, ' New York City Fire Department,'),
 (1237, ' Cantor Fitzgerald,'),
 (1241, ' Fiduciary Trust Company International,'),
 (1243, ' Cantor Fitzgerald,'),
 (1245, " New York City, Sandler O'Neill + Partners,"),
 (1246, " New York City, Sandler O'Neill + Partners,"),
 (1247, ' Cantor Fitzgerald,'),
 (1248, ' New York City, Cantor Fitzgerald,'),
 (1249, ' New York City, Fuji Bank, Ltd.,'),
 (1250, ' Lo

## Fixing a common problem.

Notice that many rows simply contain ` New York City,` without the state.  Let's fix this problem in our preprocessing step.

In [35]:
grouped_lines[41]

'David D. Alger, 57, New York City, Fred Alger Management, Inc., World Trade Center.'

In [36]:
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))
grouped_lines[41] >> fix_nyc

'David D. Alger, 57, New York City, N.Y., Fred Alger Management, Inc., World Trade Center.'

## Progress so far

In [37]:
# Imports
from composable import pipeable
from composable.strict import map

In [38]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [39]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [40]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [41]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines

d of Electrical Workers,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Darlene E. Flagg',
  ' ??',
  ' Passenger, American 77,',
  '',
  '',
  ' Pentagon',
  ''),
 ('Wilson F. Flagg',
  ' 62',
  ' Millwood, Va., Passenger, American 77,',
  '',
  '',
  ' Pentagon',
  ''),
 ('Christina Donovan Flannery',
  ' 26',
  " Middle Village, N.Y., Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Eileen Flecha',
  ' 33',
  ' Queens, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andre G. Fletcher',
  ' 37',
  ' New York City Fire Department,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Carl M. Flickinger',
  ' 38',
  ' Congers, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Matthew M. Flocco', ' ??', ' United States Navy,', '', '', ' Pentagon', ''),
 ('John Joseph Florio',
  ' 33',
  ' Oceanside, N.Y., New York City Fire Department,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Joseph Wa

In [42]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names

i',
 'Jane S. Beatty',
 'Alan Anthony Beaven',
 'Lawrence Ira Beck',
 'Manette Marie Beckles',
 'Carl John Bedigian',
 'Michael Ernest Beekman',
 'Maria A. Behr',
 'Max J. Beilke',
 'Yelena Belilovsky',
 'Nina Patrice Bell',
 'Debbie S. Bellows',
 'Stephen Elliot Belson',
 'Paul M. Benedetti',
 'Denise Lenore Benedetto',
 'Bryan Craig Bennett',
 'Eric L. Bennett',
 'Oliver Bennett',
 'Margaret L. Benson',
 'Dominick J. Berardi',
 'James Patrick Berger',
 'Steven Howard Berger',
 'John P. Bergin',
 'Alvin Bergsohn',
 'Daniel David Bergstein',
 'Graham Andrew Berkeley',
 'Michael J. Berkeley',
 'Donna M. Bernaerts',
 'David W. Bernard',
 'William H. Bernstein',
 'David M. Berray',
 'David Shelby Berry',
 'Joseph John Berry',
 'William Reed Bethke',
 'Yeneneh Betru',
 'Timothy D. Betterly',
 'Carolyn Mayer Beug',
 'Edward Frank Beyea',
 'Paul Michael Beyer',
 'Anil Tahilram Bharvaney',
 'Bella J. Bhukhan',
 'Shimmy D. Biegeleisen',
 'Peter Alexander Bielfeld',
 'William G. Biggart',
 'Bri

In [43]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit

Sandler O'Neill + Partners,",
 ' Carr Futures, Inc.,',
 ' Jersey City, N.J., Cantor Fitzgerald,',
 ' Glen Rock, N.J., Chuo Mitsui Trust and Banking Company, Ltd.,',
 ' Woodstock, N.Y., Fiduciary Trust Company International,',
 ' Summit Security Services, Inc.,',
 ' Wilmot, N.H.,',
 ' Glen Gardner, N.J., Cantor Fitzgerald,',
 ' Port Washington, N.Y., Risk Waters Group,',
 ' Staten Island, N.Y., New York City Fire Department,',
 ' Scarsdale, N.Y., Cantor Fitzgerald,',
 '',
 ' Manasquan, N.J., Cantor Fitzgerald,',
 ' Princeton Junction, N.J., Euro Brokers,',
 ' Staten Island, N.Y., New York City Fire Department,',
 ' Cantor Fitzgerald,',
 ' Norwalk, Conn., Aon Corporation visitor,',
 ' Boston, Mass. and Paris, France,',
 ' Staten Island, N.Y., Cantor Fitzgerald,',
 ' Santa Monica, Calif.,',
 ' Medford, N.Y., New York City Police Department,',
 ' Bayonne, N.J.,',
 ' Cantor Fitzgerald,',
 ' Pearl River, N.Y., New York City Police Department,',
 ' East Patchogue, N.Y., Cantor Fitzgerald,',
 

## Adding more states

Next, we will start adding start to our pattern, and again looking for additional states/problems.  For example, let's add the `Mass.` and `D.C.` patterns.

In [82]:
state = re.compile(', (N\.Y\.|N\.J\.|Mass\.|D\.C\.),?')
[(l, state.search(l)) for l in troubling_bit if state.search(l)][0:30]

[(' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 (' New York City, N.Y., Windows on the World,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object; span=(6, 13), match=', N.Y.,'>),
 (' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' New York City, N.Y., Fred Alger Management, In

In [81]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (43, ' Cantor Fitzgerald,'),
 (45, ' Cantor Fitzgerald,'),
 (51, ' Cantor Fitzgerald, Forte Food Service,'),
 (52, ' Windows on the World,'),
 (54, ' Windows on the World,'),
 (55, ' Marsh&McLennan Companies, Inc.,'),
 (56, ' Fiduciary Trust Company International,'),
 (57, ' ABM Industries Inc.,'),
 (58, ' New York City Fire Department,'),
 (59, ' Port Authority of New York and New Jersey first responders,'),
 (61, ' Port Authority Police Department,'),
 (69, ' Cantor Fitzgerald,'),
 (71, '

<h2> <font color="red"> Exercise 4.6.2 - Continue the process. </font> </h2>

Now it is your turn.  You should

1. Keep adding states to the pattern.
2. Add preprocessing steps to fix any issues.

In [88]:
state = re.compile(', (N\.Y\.|N\.J\.|New\sJersey|Mass\.|D\.C\.|DC|Ill\.|N\.M\.|NM|New Mexico|Va|Va\.|Virginia),?')
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

up conference attendee from DataSynapse,'),
 (1501, ' New York State Department of Taxation and Finance,'),
 (1507, ' Cos Cob, Conn., Keefe, Bruyette&Woods, Inc.,'),
 (1508, ' Port Authority Police Department,'),
 (1519, ' Baseline Financial Services,'),
 (1521, ''),
 (1522, ' Fairfield, Conn., Keefe, Bruyette&Woods, Inc.,'),
 (1525, ' Port Authority of New York and New Jersey,'),
 (1531, ' Forestville, Md., United States Army Civilian,'),
 (1532, ' Cantor Fitzgerald,'),
 (1537, ' Frank W. Lin&Co.,'),
 (1540, ' New York City Fire Department,'),
 (1546, ' Pitney Bowes Inc.,'),
 (1548, ' New Jersey, Washington Group International,'),
 (1555, ' Empire BlueCross BlueShield,'),
 (1557, ' United States Army,'),
 (1561, ' Fiduciary Trust Company International,'),
 (1562, ' Aramark Corporation,'),
 (1563, ''),
 (1565,
  ' Langhorne, Pa., Marsh&McLennan consultant from Devonshire Service Group, Inc.,'),
 (1566, ' Cantor Fitzgerald,'),
 (1567, " New Jersey, Sandler O'Neill + Partners,"),
 (1570,

<h2> <font color="red"> Exercise 4.6.3 - Make your solution verbose </font> </h2>

Now make your solution to the last problem verbose.  Also reorder the cases so that similar cases are close and add comments.  Finally, change the regular expression to capture the parts before and after the state.

In [47]:
# Your code here

## Splitting the troubling bit

Now that we have a way to identify rows that have home addresses (through matching the state), we will split up this data.  We will do this by considering three cases.

1. Blank entry become three blanks (for town, state, employer).
2. Lines that match the states regex will get split by this pattern.
3. The remaining lines hold only the employer and become `'','',entry`

In [48]:
def split_troubling_bit(entry):
    if len(entry) == 0:
        return ('', '', '')
    elif state.search(entry):
        return state.search(entry).groups(default='')
    else:
        return ('', '', entry)

In [49]:
( troubling_bit
 >> map(split_troubling_bit)
)

('N.Y.',),
 ('', '', ' Aon Corporation,'),
 ('', '', ' Aon Corporation,'),
 ('N.Y.',),
 ('', '', ' Alliance Consulting Group,'),
 ('', '', ' New York, Risk Waters Group,'),
 ('N.J.',),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Yardley, Pa., Aon Corporation,'),
 ('N.J.',),
 ('N.Y.',),
 ('N.Y.',),
 ('N.J.',),
 ('Mass.',),
 ('', '', ' International Office Centers Corporation,'),
 ('', '', ' Marsh&McLennan consultant,'),
 ('Mass.',),
 ('N.Y.',),
 ('N.Y.',),
 ('', '', ' Keefe, Bruyette&Woods, Inc.,'),
 ('N.J.',),
 ('', '', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Burbank, Calif., Passenger, American 77,'),
 ('N.J.',),
 ('', '', ' Santa Monica, Calif.,'),
 ('', '', ' Empire BlueCross BlueShield,'),
 ('N.Y.',),
 ('',
  '',
  ' New Jersey, Risk Waters Group conference attendee from Instinet, Inc.,'),
 ('N.J.',),
 ('N.Y.',),
 ('N.Y.',),
 ('N.Y.',),
 ('N.Y.',),
 ('N.Y.',),
 ('N.Y.',),
 ('', '', ' Tempe, Ariz., Marsh&McLennan Companies, Inc.,'),
 ('N.Y.',),
 ('', '', ' Aon Corporation

## Progress so far

In [50]:
# Imports
from composable import pipeable
from composable.strict import map

In [51]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [52]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [53]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [79]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines[0:10]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anth

In [76]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

In [80]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit[0:30]

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, N.Y., Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, N.Y., Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, N.Y., Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Manalapan, N.J., Cantor Fitzgerald,',
 ' Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Fiduciary Trust Company International,',
 ' Belle Harbor, N.Y., New York City Fire Department,',

In [57]:
state = re.compile('''
^(.*?)
,?\s                    # Optional comman
(
       N\.Y\.           
    |  N\.J\.
    |  D\.C\.
    |  N\.H\.
    |  N\.M\.
    |  N\.C\.
    |  R.I.
    |  Md\.
    |  Pa\.
    |  Va\.
    |  Ga\.
    |  La\.
    |  Mass\.
    |  Calif\.
    |  Ariz\.
    |  Fla\.
    |  Ill\.
    |  Conn\.
    |  Hawaii
    |  Iowa
    |  Maine
    |  New\sHampshire
    |  New\sJersey
    |  New\sYork
    |  Ohio
    |  Pennsylvania
    |  Texas
    |  Utah
    |  Virginia
    |  Japan
    |  India
    |  Germany
    |  Manitoba,\sCanada
    |  New\sSouth\sWales,\sAustralia
    |  England,\sUnited\sKingdom
)
,
(.*?)$
''', re.VERBOSE)

In [58]:
( troubling_bit
 >> map(split_troubling_bit)
)

' Cantor Fitzgerald,'),
 (' Alexandria', 'Va.', ' United States Navy,'),
 (' Mohegan Lake', 'N.Y.', ' ABM Industries Inc.,'),
 (' Staten Island', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Framingham', 'Mass.', ''),
 (' Fresh Meadows', 'N.Y.', " Sandler O'Neill + Partners,"),
 (' Brooklyn', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Bronx', 'N.Y.', ' New York City Fire Department,'),
 (' Allendale', 'N.J.', ' Keefe, Bruyette&Woods, Inc.,'),
 (' Staten Island', 'N.Y.', ' ABM Industries Inc.,'),
 ('', 'New York', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Aon Corporation,'),
 (' Long Island City', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Aon Corporation,'),
 (' Ledgewood', 'N.J.', ''),
 (' New York City', 'N.Y.', ' Aon Corporation,'),
 (' Farmingville', 'N.Y.', ' New York City Fire Department,'),
 ('', '', ' Marsh&McLennan consultant,'),
 (' New York City', 'N.Y.', ' Fiduciary Trust Company International,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' 

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Clean up each part of the troubling bits, then comma join this section into 1 string.

**Hint:** Be sure to remove any problematic commas.

In [59]:
# Your code here

## Combining the parts back together.

We can combine the parts back together using the `zip` function.

In [60]:
from composable.strict import zipOnto
from composable.list import to_list
(zip(names, ages, fixed_troubling_bits)
 >> to_list
 >> map(comma_join)
)

NameError: name 'fixed_troubling_bits' is not defined

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Use `zip` to combine all part of the data and write the result out to a file called `911_Deaths_Fixed.csv` 

In [539]:
# Your code here